# Julia 機器學習：LIBSVM 支撐向量機

## 作業 029：

請使用 SVM 來建立一個預測模型，來對手寫數字辨識資料集，建立一個分類模型，預測數字為何。

本作業需要使用到的套件有 MLDatasets，請在執行以下範例前先安裝。

```
] add MLDatasets
```

In [3]:
using LIBSVM, MLDatasets, Statistics

┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1273


## 讀取資料

呼叫 MNIST 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。

In [4]:
train_X, train_y = MNIST.traindata(1:10000)
test_X,  test_y  = MNIST.testdata(1:1000)

This program has requested access to the data dependency MNIST.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: THE MNIST DATABASE of handwritten digits
Authors: Yann LeCun, Corinna Cortes, Christopher J.C. Burges
Website: http://yann.lecun.com/exdb/mnist/

[LeCun et al., 1998a]
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner.
    "Gradient-based learning applied to document recognition."
    Proceedings of the IEEE, 86(11):2278-2324, November 1998

The files are available for download at the offical
website linked above. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of MNIST aren't really
explicit about any terms of use, so please read the
website to make sure you want to download the
dataset.



Do you want to download the dataset from ["http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyt

(FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

...

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N

In [6]:
@show typeof(train_X)

typeof(train_X) = Base.ReinterpretArray{FixedPointNumbers.Normed{UInt8,8},3,UInt8,Array{UInt8,3}}


Base.ReinterpretArray{FixedPointNumbers.Normed{UInt8,8},3,UInt8,Array{UInt8,3}}

In [13]:
@show size(train_X)
@show size(test_X)

@show size(train_y)
@show size(test_y)

size(train_X) = (28, 28, 10000)
size(test_X) = (28, 28, 1000)
size(train_y) = (10000,)
size(test_y) = (1000,)


(1000,)

## 資料轉換

In [14]:
train_X = Matrix(reshape(train_X, (10000, 28*28)))
test_X = Matrix(reshape(test_X, (1000, 28*28)))

1000×784 Array{N0f8,2} with eltype FixedPointNumbers.Normed{UInt8,8}:
 0.0    0.0  0.0    0.0    0.0    0.0    …  0.0    0.0    0.0    0.0    0.0  
 0.0    0.0  0.671  0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.502
 0.0    0.0  0.996  0.0    0.0    0.408     0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.318  0.0    0.0    0.996     0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.0    0.0    0.62      0.173  0.0    0.0    0.992  0.988
 0.0    0.0  0.0    0.0    0.0    0.0    …  0.898  0.0    0.0    0.984  0.992
 0.0    0.0  0.0    0.282  0.0    0.0       0.992  0.0    0.0    0.984  0.988
 0.0    0.0  0.0    0.682  0.0    0.0       0.494  0.0    0.0    0.122  0.988
 0.0    0.0  0.0    0.984  0.0    0.0       0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.678  0.0    0.0       0.0    0.0    0.0    0.0    0.992
 0.0    0.0  0.0    0.278  0.0    0.0    …  0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.282  0.0    0.0       0.0    0.0    0.0    0.0    

In [10]:
train_y = Vector(train_y)
test_y = Vector(test_y)

1000-element Array{Int64,1}:
 7
 2
 1
 0
 4
 1
 4
 9
 5
 9
 0
 6
 9
 ⋮
 1
 6
 2
 3
 9
 0
 1
 2
 2
 0
 8
 9

## 預測

In [16]:
model = LIBSVM.fit!(SVC(), train_X, train_y)

SVC(LIBSVM.Kernel.RadialBasis, 0.0012755102040816326, nothing, 1.0, 3, 0.0, 0.001, true, false, false, LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 784, 10, [5, 0, 4, 1, 9, 2, 3, 6, 7, 8], Int32[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], Float64[], Int32[], LIBSVM.SupportVectors{Int64,FixedPointNumbers.Normed{UInt8,8}}(9999, Int32[863, 1001, 980, 1126, 978, 991, 1032, 1014, 1070, 944], [5, 5, 5, 5, 5, 5, 5, 5, 5, 5  …  8, 8, 8, 8, 8, 8, 8, 8, 8, 8], FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.498N0f8 … 0.671N0f8 0.0N0f8; … ; 0.816N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8], Int32[1, 12, 36, 48, 66, 101, 133, 139, 146, 174  …  9936, 9937, 9938, 9945, 9949, 9953, 9957, 9971, 9983, 9997], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNod

In [25]:
test_X[1, :]

784-element Array{N0f8,1} with eltype FixedPointNumbers.Normed{UInt8,8}:
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.329N0f8
 0.588N0f8
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 ⋮        
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  
 0.0N0f8  

In [27]:
ŷ = String[]

for i = 1:1000
    y_ = LIBSVM.predict(model, test_X[i, :])
    append!(ŷ, y_)
end

ErrorException: Model has 784 but 1 provided

## 評估